In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd

In [2]:
# Import the CPM data
cpm = pd.read_csv('data/cpm_estimates-15Apr19.csv')
cpm.head()

,age_min,age_max,female,male,cpm,hID,date
0,18,24,0,1,3.678763,8910733059390721166,2018-10-02
1,18,24,1,0,3.026950,8910733059390721166,2018-10-02
2,18,24,1,1,3.114728,8910733059390721166,2018-10-02
3,25,34,0,1,3.011565,8910733059390721166,2018-10-02
4,25,34,1,0,4.032152,8910733059390721166,2018-10-02


In [3]:
# Clean the CPM data for EDA purposes
def clean_cpm(data):
    
    # Extract demographic features
    data['age_group'] = data['age_min'].astype('str') + '-' + data['age_max']
    data.loc[data['female'] == 1, 'gender'] = 'female'
    data.loc[data['male'] == 1, 'gender'] = 'male'
    data.loc[(cpm['female'] == 1) & (data['male'] == 1), 'gender'] = 'unisex'
    data['demographic'] = data['age_group'] + ' ' + data['gender']
    data['date'] = pd.to_datetime(data['date'])

    # Extract additional time features
    data['year'] = data.date.dt.year
    data['month'] = data.date.dt.month
    data['day'] = data.date.dt.day

    # Clean up the data for later Machine Learning
    data['age_max'].replace({'65+':'65'},inplace=True)
    data['age_max'] = data['age_max'].astype('int')
    data['cpm'].replace({0:np.nan},inplace=True)
    data['cpm'].fillna(value=data['cpm'].median(),inplace=True)
    
    return data

# Create a new dataframe called cpm_eda
cpm = clean_cpm(cpm)
cpm.head()

,age_min,age_max,female,male,cpm,hID,date,age_group,gender,demographic,year,month,day
0,18,24,0,1,3.678763,8910733059390721166,2018-10-02,18-24,male,18-24 male,2018,10,2
1,18,24,1,0,3.026950,8910733059390721166,2018-10-02,18-24,female,18-24 female,2018,10,2
2,18,24,1,1,3.114728,8910733059390721166,2018-10-02,18-24,unisex,18-24 unisex,2018,10,2
3,25,34,0,1,3.011565,8910733059390721166,2018-10-02,25-34,male,25-34 male,2018,10,2
4,25,34,1,0,4.032152,8910733059390721166,2018-10-02,25-34,female,25-34 female,2018,10,2


In [9]:
# Import the page level data
files = [i for i in os.listdir('data') if 'csv' in i and 'sample' not in i and i.startswith('page')]

# Concatenate all the imported data from the iteration
page_metrics = pd.DataFrame()
for file in files:
    temp = pd.read_csv('data/'+file)
    page_metrics = pd.concat([page_metrics,temp])

# Filter the page level data to only include those with demographic features
page_demo = page_metrics[page_metrics.name == 'page_impressions_by_age_gender_unique'].copy()
page_demo.date = pd.to_datetime(page_demo.date)

# Reduce the page_metrics data to have hIDs that have demographic features for later pipelining
print(f'page_demo has {page_demo.hID.nunique()} unique hIDs')
print(f'page_metrics has {page_metrics.hID.nunique()} unique hIDs')

# Reduce page_metrics data to only include 444 hIDs that have demographic metrics data
page_metrics = page_metrics[page_metrics.hID.isin(page_demo.hID.unique())]
print(f'page_metrics NOW has {page_metrics.hID.nunique()} unique hIDs')

page_demo has 444 unique hIDs
page_metrics has 597 unique hIDs
page_metrics NOW has 444 unique hIDs


In [5]:
page_metrics.name.unique()

array(['page_fans_online', 'page_views', 'page_impressions_organic',
       'page_video_views_paid', 'page_post_engagements',
       'page_impressions_paid', 'page_posts_impressions_paid',
       'page_cta_clicks_logged_in_total', 'page_impressions',
       'page_posts_impressions_organic', 'page_video_views',
       'page_posts_impressions', 'page_views_total',
       'page_video_views_organic', 'page_impressions_by_city_unique',
       'page_impressions_by_locale_unique',
       'page_impressions_by_country_unique',
       'page_impressions_by_age_gender_unique',
       'page_views_by_age_gender_logged_in_unique'], dtype=object)

In [6]:
page_metrics = page_metrics.pivot_table(values='value', index='hID', columns='name')[['page_impressions', 'page_cta_clicks_logged_in_total']]

In [7]:
# Prepare the page_demo data for CPM imputations
def clean_page_demo(data):

    '''
    1. Extract the demographic information such as age and gender
    2. Extract time features such as year and month
    3. Match the feature to the random forest model inputs
    '''
    
    data['age_group'] = [i[2:] for i in data['metric']]
    data['age_group'].replace({'65+':'65-85'},inplace=True)
    data['age_min'] = [i[:2] for i in data['age_group']]
    data['age_max'] = [i[-2:] for i in data['age_group']]
    data['female'] = [1 if i[0] == 'F' or i[0] == 'U' else 0 for i in data['metric']]
    data['male'] = [1 if i[0] == 'M' or i[0] == 'U' else 0 for i in data['metric']]
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['day'] = data['date'].dt.day
    data.drop('name',axis=1,inplace=True)
    data.rename({'value':'impressions'},axis=1,inplace=True)

    return data

page_demo = clean_page_demo(page_demo)
page_demo.head()

,date,metric,impressions,hID,age_group,age_min,age_max,female,male,year,month,day
4640,2019-01-01,U.35-44,4.0,3141727685331597704,35-44,35,44,1,1,2019,1,1
4642,2019-01-02,M.25-34,242.0,3141727685331597704,25-34,25,34,0,1,2019,1,2
4645,2019-01-06,F.55-64,67.0,3141727685331597704,55-64,55,64,1,0,2019,1,6
4647,2019-01-01,F.55-64,133.0,3141727685331597704,55-64,55,64,1,0,2019,1,1
4653,2019-01-03,M.45-54,86.0,3141727685331597704,45-54,45,54,0,1,2019,1,3


In [8]:
cpm.to_csv('clean_data/cpm.csv',index=False)
page_demo.to_csv('clean_data/page_demo.csv',index=False)
page_metrics.to_csv('clean_data/page_metrics.csv',index=False)